# Draft analysis 

---

Group name: Gruppe A | Ardit Fazliu, Lukas Laib, Marcus Wild

---


## Introduction
*This section includes an introduction to the project motivation, data and research question. Include a data dictionary.*

### Subject

Der Vertrieb und Verkauf von Waren über das Internet ist zu einem wichtigen Vertriebskanal der modernen, digitalisierten Wirtschaft geworden. Diese Entwicklung wurde durch Covid19-Pandemie weiter verstärkt. So stiegen in Deutschland die im B2C E-Commerce erzielten Umsätze im Jahr 2020 gegenüber dem Vorjahr um fast 23% auf knapp 73 Milliarden Euro (Statista, 2022a). Auch innerhalb des B2B E-Commerce wird in den kommenden Jahren ein enormes Umsatzwachstum erwartet, in den USA wird der Umsatz im Jahr 2025 auf 2,5 Billionen steigen, dies entspricht einer Wachstumsrate von rund 11% p.a. (Statista, 2022b). 

### Motivation

Während die Umsätze im E-Commerce Jahr für Jahr steigen, sind die Margen im Onlinehandel für einen Großteil der Händler marginal. Dennoch müssen Unternehmen hohe Investitionen in das Online-Geschäft tätigen, um Umsätze im stark umkämpften Online-Geschäft zu erzielen (Alvarez & Marsal, 2021). Damit sich diese nachhaltig für das Unternehmen lohnen, müssen langfristig verlässliche Gewinne erwirtschaftet werden. Dieses Ziel kann nur erreicht werden, wenn externe Risiken für das Unternehmen minimiert und intern strategisch richtige Entscheidungen zur Portfolioerweiterung getroffen werden. Im folgenden soll ein Datensatz, bestehend aus Transaktionsdaten eines E-Commerce Unternehmen aus dem Vereinigten Königreich analysiert werden, um Abhängigkeiten von bestimmten  1.) Ländern oder Regionen 2) Kunden 3) Produkten bzw. Produktgruppen zu identifizieren und 4) Potenziale für Umsatzwachstum / Gewinnsteigerung zu entdecken.

### General Research Question

**Wie können Abhängigkeiten für den E-Commerce Shop langfristig reduziert werden und welche Maßnahmen können zur langfristigen Umsatzmaximierung getroffen werden?**

### Data Description

Der Datensatz zeigt Transaktionsdaten eines E-Commerce Unternehmen aus dem Vereinigten Koenigreich. Die Daten wurden zwischen dem 01.12.2010 und dem 09.12.2011 erfasst.

Jeder Beobachtungswert stellt innerhalb des Datensatzes eine Transaktion dar, wobei mehreren Transaktionen eine Rechnung(snummer) zugeteilt werden kann.

Der Datensatz enthaelt in der ursprünglichen Versionierung 541909 Beobachtungswerte und 8 Spalten.

### Data Dictionary
*Das Skalenniveau ist detaillierter angegeben. Statt "Numeric" wurde die Unterteilung zwischen intervall- und verhaeltnisskaliert verwendet.

| Spaltenname  | Beschreibung  | Skalenniveau | Format |
|---|---|---|---|
|InvoiceNo | Rechnungsnummer, 6-stellige Nummer, welche jeder Transaktion eindeutig zuordnet. <br>Falls die Nummer ein "c" enthält wurde die Transaktion storniert. | Nominal | object |
|StockCode | Produktnummer, 5-stellige Nummer, welche jedem einzelnem Produkt eindeutig zugeordnet ist. | Nominal | object |
|Description | Beschreibung des Produkts. | Nominal | object |
|Quantity | Menge der einzelnen Produkte pro Transaktion. | Verhaeltnis | int |
|InvoiceDate | Uhrzeit und Datum der Rechnung. | Intervall | date |
|UnitPrice | Preis pro Produkt. | Verhaeltnis | float |
|CustomerID | Kundennummer, 5-stellige Nummer, welche je einem Kunden eindeutig zugeordnet ist. | Nominal | object |
|Country | Name des Landes in welchem der Kunde ansässig ist. | Nominal | category |
|TotalPrice | Produkt aus Quantity und UnitPrice. | Verhaeltnis | float |

## Setup

In [77]:
# Setup
# Import relevanter Python-Bibliotheken

import pandas as pd
import altair as alt

## Data

### Description of the relevant variables



- **InvoiceNo:** Rechnungsnummer, welche jeder Transaktion eindeutig zuordnet.
- **StockCode:** Produktnummer, welche jedem Produkt eindeutig zuordnet.
- **Description:** Beschreibung des Produkts.
- **Quantity:** Menge der bestellten Produkte pro Transaktion.
- **InvoiceDate:** Uhrzeit und Datum der Rechnung.
- **UnitPrice:** Preis pro Produkt.
- **CustomerID:** Kundennummer, welche jedem Kunden eindeutig zuordnet.
- **Country:** Name des Landes in welchem der Kunde ansässig ist.
- **TotalPrice:** Gesamtwert der Transaktion.
<br><br>

| Nummer | (Alternativ-)Hypothese  | Relevante Variablen |
|---|---|---|
| H1 | Mit einem Kunden wird ein Umsatz von mindestens zehn Prozent des Gesamtumsatzes erzielt. | CustomerID, TotalPrice |
| H2 | Mit einem Produkt wird ein Umsatz von mindestens zehn Prozent des Gesamtumsatzes erzielt. | StockCode, TotalPrice |
| H3 | Der erzielte Umsatz innerhalb eines Landes beträgt mindestens 40%. | Country, TotalPrice |
| H4 | Welchen Einfluss hat der Kundenstandort auf den Warenkorbwert. | Country, InvoiceNo, TotalPrice |
| H5 | Welchen Einfluss hat die Weihnachtssaison auf die verkauften Produkte. | InvoiceDate, StockCode |

### Import data

In [78]:
# Import des Datensatzes
raw_data = "https://raw.githubusercontent.com/ArditFazliu/Analytics-und-Data-Storytelling/main/data/raw_data.csv" 
df = pd.read_csv(raw_data, encoding= 'unicode_escape')

### Data structure

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Data corrections

##### Anpassung der Datentypen

In [80]:
# Erstellen einer neuen Variable, um den gesamten Preis pro Transaktion zu ermitteln
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [81]:
# Anpassung des Datums
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format="%m/%d/%Y %H:%M")

In [82]:
# Anpassung der kategorialen Variablen
df = df.astype(
    {
        "InvoiceNo": "category",
        "StockCode": "category",
        "CustomerID": "category",
        "Country": "category"

})

##### Fehlende Werte

In [83]:
#Fehlende Werte auslesen
df.isna().sum()
#Fehlende Werte löschen
df.dropna()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [84]:
non_stockcode = []
for s in df["StockCode"]:
    if any(i.isdigit() for i in s) == False:
        non_stockcode.append(s)

In [85]:
pd.value_counts(non_stockcode)

POST            1256
DOT              710
M                571
D                 77
S                 63
BANK CHARGES      37
AMAZONFEE         34
CRUK              16
DCGSSGIRL         13
DCGSSBOY          11
PADS               4
B                  3
m                  1
dtype: int64

In [86]:
set_stockcode = set(non_stockcode)
df = df[~df["StockCode"].isin(set_stockcode)]

### Findings zu missing values?


## Analysis

### Descriptive statistics

In [87]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539113 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    539113 non-null  category      
 1   StockCode    539113 non-null  category      
 2   Description  537664 non-null  object        
 3   Quantity     539113 non-null  int64         
 4   InvoiceDate  539113 non-null  datetime64[ns]
 5   UnitPrice    539113 non-null  float64       
 6   CustomerID   405043 non-null  category      
 7   Country      539113 non-null  category      
 8   TotalPrice   539113 non-null  float64       
dtypes: category(4), datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 29.8+ MB


In [89]:
# Überblick der nummerischen Variablen
df.describe()

,Quantity,UnitPrice,TotalPrice
count,539113.000000,539113.000000,539113.000000
mean,9.583215,3.282171,18.178166
std,218.591696,4.560479,367.046393
min,-80995.000000,0.000000,-168469.600000
25%,1.000000,1.250000,3.400000
50%,3.000000,2.080000,9.360000
75%,10.000000,4.130000,17.400000
max,80995.000000,649.500000,168469.600000


#Findings zu Descriptive statistics ?

In [90]:

# Überblick des Rechnungsdatums
df.describe(include="datetime64[ns]")

/var/folders/df/nc1vtthj247d1h5l5689kdxh0000gn/T/ipykernel_47691/1977976457.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include="datetime64[ns]")


,InvoiceDate
count,539113
unique,22775
top,2011-10-31 14:41:00
freq,1113
first,2010-12-01 08:26:00
last,2011-12-09 12:50:00


In [91]:


# Überblick der kategorialen Variablen
df.describe(include=[object,"category"])

,InvoiceNo,StockCode,Description,CustomerID,Country
count,539113,539113,537664,405043.0,539113
unique,25303,4057,4211,4363.0,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,17841.0,United Kingdom
freq,1113,2313,2369,7971.0,493890


### Exploratory data analysis

#### H1: Mit einem Kunden wird ein Umsatz von mindestens zehn Prozent des Gesamtumsatzes erzielt.

##### Die 10 umsatzstärksten Kunden

In [92]:
df_sales = df[df["Quantity"] > 0]


In [93]:

# Filter nach den umsatzstärksten Kunden
df_plot = df_sales.groupby("CustomerID").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["CustomerID"] = df_plot.index


In [94]:

# Visualisierung der umsatzstärksten Kunden
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("CustomerID",
            sort="-x", 
            axis=alt.Axis(title = "Kundennummer",
                          grid=False,
                          labelAngle=0,
                          titleAnchor="middle"

                          ))
).properties(
    title="Die 10 umsatzstärksten Kunden",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title( 
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
    
    )

alt.Chart(...)

##### Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach Umsatz

In [95]:
# Umsatz der 75 umsatzstärksten Kunden
df_plot = df_sales.groupby("CustomerID").sum().sort_values(["TotalPrice"], ascending=False)[:75]
df_plot["CustomerID"] = df_plot.index

# Visualisierung der Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice:Q",
            bin=alt.BinParams(maxbins=20),
            axis=alt.Axis(title="Umsatz pro Kunde in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("count()",
            axis=alt.Axis(title = "Anzahl an Kunden",
                          titleAnchor="middle",
                          grid=False))
).properties(
    title="Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach Umsatz",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title( 
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
    
)

alt.Chart(...)

#### H2: Mit einem Produkt wird ein Umsatz von mindestens zehn Prozent des Gesamtumsatzes erzielt.

##### 10 umsatzstärkste Produkte

In [96]:
# Filter nach den umsatzstärksten Produkten
df_plot = df_sales.groupby("StockCode").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["StockCode"] = df_plot.index

# Visualisierung der umsatzstärksten Kunden
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("StockCode",
            sort="-x", 
            axis=alt.Axis(title = "Produktnummer",
                          titleAnchor="middle"))
).properties(
    title="10 umsatzstärkste Produkte",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18

)

alt.Chart(...)

##### Häufigkeitsverteilung der 75 umsatzstärksten Produkte

In [97]:
# Umsatz der 75 umsatzstärksten Produkte
df_plot = df_sales.groupby("StockCode").sum().sort_values(["TotalPrice"], ascending=False)[:75]
df_plot["StockCode"] = df_plot.index

# Visualisierung der Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice:Q",
            bin=alt.BinParams(maxbins=20),
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("count()",
            axis=alt.Axis(title = "Anzahl an Produkten",
                          titleAnchor="middle",
                          grid=False))
).properties(
    title="Häufigkeitsverteilung der 75 umsatzstärksten Produkte",
    width=550,
    height=250
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

#### H3: Der erzielte Umsatz innerhalb eines Landes beträgt mindestens 40%.

##### Zehn umsatzstärksten Länder

In [98]:
# Filter nach den umsatzstärksten Ländern
df_sales = df_sales[~df_sales["StockCode"].isin(set_stockcode)]
df_plot = df_sales.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["Country"] = df_plot.index


# Visualisierung der umsatzstärksten Kunden
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("Country",
            sort="-x", 
            axis=alt.Axis(title = "Land",
                          titleAnchor="middle"))
).properties(
    title="Zehn umsatzstärksten Länder",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

##### Zehn umsatzstärksten Länder ohne UK

In [99]:
# Filter nach den umsatzstärksten Ländern ohne UK
df_sales = df_sales[~df_sales["StockCode"].isin(set_stockcode)]
df_plot = df_sales.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[1:11]
df_plot["Country"] = df_plot.index


# Visualisierung der umsatzstärksten Kunden
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("Country",
            sort="-x", 
            axis=alt.Axis(title = "Land",
                          titleAnchor="middle"))
).properties(
    title="Zehn umsatzstärksten Länder ohne UK",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=14
)

alt.Chart(...)

##### Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz

In [100]:
# Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
df_plot = df_sales.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[:75]
df_plot["Country"] = df_plot.index

# Visualisierung der Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice:Q",
            bin=alt.BinParams(maxbins=60),
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("count()",
            axis=alt.Axis(title = "Anzahl an Ländern",
                          titleAnchor="middle",
                          grid=False))
).properties(
    title="Häufigkeitsverteilung der Absatzländer nach Umsatz",
    width=500,
    height=200
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

##### Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz ohne Uk

In [101]:
# Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
df_plot = df_sales.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[1:76]
df_plot["Country"] = df_plot.index

# Visualisierung der Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice:Q",
            bin=alt.BinParams(maxbins=60),
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("count()",
            axis=alt.Axis(title = "Anzahl an Ländern",
                          titleAnchor="middle",
                          grid=False))
).properties(
    title="Häufigkeitsverteilung der Absatzländer nach Umsatz",
    width=500,
    height=200
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

#### H4: Welchen Einfluss hat der Kundenstandort auf den Warenkorbwert.

##### Top 10 Länder mit dem höchsten Warenkorbwert

In [102]:
#Dataframe mit Gesamtumsatz gruppiert für InvoiceNo und Country
df_plot = df_sales.groupby(["Country"]).sum().sort_values(["TotalPrice"], ascending=False)
#Dataframe mit Anzahl der Rechnungsnummern pro Land
df_plot2 = df_sales.groupby(["Country"]).count().sort_values(["InvoiceNo"], ascending=False)
#Dataframe mit Spalte zur Berechnung des Warenkorbwertes
df_plot["TotalPrice/InvoiceNo"] = df_plot["TotalPrice"]/df_plot2["InvoiceNo"]
df_plot = df_plot.sort_values("TotalPrice/InvoiceNo", ascending=False)[:10]
df_plot["Country"]= df_plot.index

In [103]:
#Visualisierungen der zehn Länder mit dem höchsten Warenkorbwert
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice/InvoiceNo",
            axis=alt.Axis(title="Warenkorbwert in £",
                            labelAngle=0,
                            titleAnchor="start",
                            grid=False)),
    y=alt.Y("Country",
            sort="-x",
            axis=alt.Axis(title ="Land",
                            titleAnchor="middle"))
).properties(
    title="Top 10 Länder mit dem höchsten Warenkorbwert",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

#### H5: Welchen Einfluss hat die Weihnachtssaison auf die verkauften Produkte.

##### Umsatz pro Monat | Dezember 2010 - Dezember 2011

In [104]:
df_sales["Year"] = df_sales.InvoiceDate.dt.year
df_sales["Month"] = df_sales.InvoiceDate.dt.month
df_sales["Weekday"] = df_sales.InvoiceDate.dt.weekday
df_sales["Day"] = df_sales.InvoiceDate.dt.day
df_plot = df_sales.groupby(["Year","Month"]).sum().sort_values(["Year","Month"], ascending=True)
df_plot["Year"] = df_plot.index.get_level_values(0)
df_plot["Month"] = df_plot.index.get_level_values(1)
df_plot = df_plot.reset_index(drop=True)
df_plot["Year_Month"] = df_plot["Year"].astype(str) + "-" + df_plot["Month"].astype(str)
df_plot["Year_Month"] = df_plot["Year_Month"].str[2:]

In [105]:
# Visualisierung der umsatzstärksten Monate
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("Year_Month",
            sort = ["10-12","11-1","11-2","11-3","11-4"],
            axis=alt.Axis(title="Monat", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("TotalPrice",
            sort="-x", 
            axis=alt.Axis(title = "Umsatz",
                          titleAnchor="middle"))
).properties(
    title="Umsatz pro Monat | Dezember 2010 - Dezember 2011",
    width=750,
    height=350
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

##### Umsatz pro Tag | Dezember 2010

In [106]:
df_plot = df_sales.loc[(df_sales["Year"] == 2010) & (df_sales["Month"] == 12)]
df_plot = df_plot.groupby("Day").sum()
df_plot["Day"]= df_plot.index

In [107]:
# Visualisierung der Umsätze pro Tag im Dezember 2010
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("Day",
            axis=alt.Axis(title="Dezember 2010", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("TotalPrice", 
            axis=alt.Axis(title = "Umsatz",
                          titleAnchor="middle"))
).properties(
    title="Umsatz pro Tag | Dezember 2010",
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)

alt.Chart(...)

##### Umsatz pro Tag | Dezember 2011


In [108]:
# Visualisierung der Umsätze pro Tag im Dezember 2011
alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("Day",
            axis=alt.Axis(title="Dezember 2011", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("TotalPrice", 
            axis=alt.Axis(title = "Umsatz",
                          titleAnchor="middle"))
).properties(
    title="Umsatz pro Tag | Dezember 2011",
    width=500,
    height=200
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
    
)

alt.Chart(...)

## Visualizations

### Visualization ideas
**H1:** Balkendiagramm (Vergleich der Umsätze zwischen mehreren Kunden), Kreisdiagramm (Vergleich zwischen einem Kunden und den restlichen Kunden), Liniendiagramm (Verlauf der Bestellung eines oder aggregiert weniger einflussreicher Kunden).<br><br>
**H2:** Balkendiagramm (Vergleich der Umsätze zwischen mehreren Produkten), Kreisdiagramm (Vergleich zwischen einem Produkt und den restlichen Produkten), Liniendiagramm (Verlauf der Bestellung eines oder aggregiert mehrerer Produkte).<br><br>
**H3:** Balkendiagramm (Vergleich der Umsätze zwischen mehreren Ländern), Kreisdiagramm (Vergleich zwischen einem Land und den restlichen Ländern), Liniendiagramm (Verlauf der Bestellung eines oder aggregiert mehrerer Produkte), (Welt- /Europa-)Karte mit Ländern und deren Umsätze. Visualisierungen wären generell je für Land oder Region möglich.<br><br>
**H4:** Balkendiagramm (Median und Durchschnitt des Warenkorbes je Land im Vergleich zu den weiteren Ländern), Karte mit Ländern und deren Warenkorbwert, Veränderung über die Zeit / bspw. Monate, Tage - mögliche Darstellung Liniendiagramm bzw. Balkendiagramm.<br><br>
**H5:** Balken-/ Liniendiagramm der Verkäufe in Tagen, Liniendiagramm zur Veranschaulichung des Verlaufs von Produkten, Wordcloud (mit Produktbeschreibungen) für unterschiedliche Zeiträume bzw. häufige Worte integriert in ein Balkendiagramm zum besseren Vergeleich von Unterschieden.

In [109]:
# Filter nach den umsatzstärksten Kunden
df_plot = df_sales.groupby("CustomerID").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["CustomerID"] = df_plot.index

# Visualisierung der umsatzstärksten Kunden
chart_c1 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("CustomerID",
            sort="-x", 
            axis=alt.Axis(title = "Kundennummer",
                          grid=False,
                          labelAngle=0,
                          titleAnchor="middle"

                          ))
).properties(
    title={"text":["Die 10 umsatzstärksten Kunden"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title( 
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
    
)
chart_c1

alt.Chart(...)

In [110]:
# Umsatz der 75 umsatzstärksten Kunden
df_plot = df_sales.groupby("CustomerID").sum().sort_values(["TotalPrice"], ascending=False)[:75]
df_plot["CustomerID"] = df_plot.index

# Visualisierung der Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
chart_c2 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice:Q",
            bin=alt.BinParams(maxbins=20),
            axis=alt.Axis(title="Umsatz pro Kunde in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("count()",
            axis=alt.Axis(title = "Anzahl an Kunden",
                          titleAnchor="middle",
                          grid=False))
).properties(
    title={"text":["Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach Umsatz"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title( 
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
    
)
chart_c2

alt.Chart(...)

In [111]:
# Filter nach den umsatzstärksten Produkten
df_plot = df_sales.groupby("StockCode").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["StockCode"] = df_plot.index

# Visualisierung der umsatzstärksten Kunden
chart_p1 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("StockCode",
            sort="-x", 
            axis=alt.Axis(title = "Produktnummer",
                          titleAnchor="middle"))
).properties(
    title={"text":["Die 10 umsatzstärksten Produkte"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18

)
chart_p1

alt.Chart(...)

In [112]:
# Umsatz der 75 umsatzstärksten Produkte
df_plot = df_sales.groupby("StockCode").sum().sort_values(["TotalPrice"], ascending=False)[:75]
df_plot["StockCode"] = df_plot.index

# Visualisierung der Häufigkeitsverteilung der 75 umsatzstärksten Kunden nach dem Umsatz
chart_p2 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice:Q",
            bin=alt.BinParams(maxbins=20),
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("count()",
            axis=alt.Axis(title = "Anzahl an Produkten",
                          titleAnchor="middle",
                          grid=False))
).properties(
    title={"text":["Häufigkeitsverteilung der 75 umsatzstärksten Produkte nach Umsatz"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=250
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)
chart_p2

alt.Chart(...)

In [113]:
# Filter nach den umsatzstärksten Ländern
df_sales = df_sales[~df_sales["StockCode"].isin(set_stockcode)]
df_plot = df_sales.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["Country"] = df_plot.index


# Visualisierung der umsatzstärksten Kunden
chart_co1 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("Country",
            sort="-x", 
            axis=alt.Axis(title = "Land",
                          titleAnchor="middle")),
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["United Kingdom"]),
        alt.value("orange"),     
        alt.value("steelblue")     
    )
).properties(
    title={"text":["Die 10 umsatzstärksten Länder"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)
chart_co1

alt.Chart(...)

In [114]:
# Filter nach den umsatzstärksten Ländern ohne UK
df_sales = df_sales[~df_sales["StockCode"].isin(set_stockcode)]
df_plot = df_sales.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[1:11]
df_plot["Country"] = df_plot.index


# Visualisierung der umsatzstärksten Kunden
chart_co2 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice",
            axis=alt.Axis(title="Umsatz in £", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("Country",
            sort="-x", 
            axis=alt.Axis(title = "Land",
                          titleAnchor="middle")),
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["Australia","Japan"]),
        alt.value("orange"),     
        alt.value("steelblue")     
    )
).properties(
    title={"text":["Die 10 umsatzstärksten Länder ohne UK"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=14
)
chart_co2

alt.Chart(...)

In [115]:
#Dataframe mit Gesamtumsatz gruppiert für InvoiceNo und Country
df_plot = df_sales.groupby(["Country"]).sum().sort_values(["TotalPrice"], ascending=False)
#Dataframe mit Anzahl der Rechnungsnummern pro Land
df_plot2 = df_sales.groupby(["Country"]).count().sort_values(["InvoiceNo"], ascending=False)
#Dataframe mit Spalte zur Berechnung des Warenkorbwertes
df_plot["TotalPrice/InvoiceNo"] = df_plot["TotalPrice"]/df_plot2["InvoiceNo"]
df_plot = df_plot.sort_values("TotalPrice/InvoiceNo", ascending=False)[:10]
df_plot["Country"]= df_plot.index

In [116]:
#Visualisierungen der zehn Länder mit dem höchsten Warenkorbwert
chart_pv1 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("TotalPrice/InvoiceNo",
            axis=alt.Axis(title="Warenkorbwert in £",
                            labelAngle=0,
                            titleAnchor="start",
                            grid=False)),
    y=alt.Y("Country",
            sort="-x",
            axis=alt.Axis(title ="Land",
                            titleAnchor="middle")),
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["Australia","Japan"]),
        alt.value("orange"),     
        alt.value("steelblue")     
    )
).properties(
    title={"text":["Top 10 Länder mit dem höchsten Warenkorbwert"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)
chart_pv1

alt.Chart(...)

In [117]:
df_sales["Year"] = df_sales.InvoiceDate.dt.year
df_sales["Month"] = df_sales.InvoiceDate.dt.month
df_sales["Weekday"] = df_sales.InvoiceDate.dt.weekday
df_sales["Day"] = df_sales.InvoiceDate.dt.day
df_plot = df_sales.groupby(["Year","Month"]).sum().sort_values(["Year","Month"], ascending=True)
df_plot["Year"] = df_plot.index.get_level_values(0)
df_plot["Month"] = df_plot.index.get_level_values(1)
df_plot = df_plot.reset_index(drop=True)
df_plot["Year_Month"] = df_plot["Year"].astype(str) + "-" + df_plot["Month"].astype(str)
df_plot["Year_Month"] = df_plot["Year_Month"].str[2:]

In [118]:
# Visualisierung der umsatzstärksten Monate
chart_t1 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("Year_Month",
            sort = ["10-12","11-1","11-2","11-3","11-4"],
            axis=alt.Axis(title="Monat", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("TotalPrice",
            sort="-x", 
            axis=alt.Axis(title = "Umsatz",
                          titleAnchor="middle"))
).properties(
    title={"text":["Umsatz pro Monat"], "subtitle":["Im Zeitraum vom Dezember 2010 - Dezember 2011"]},
    width=750,
    height=350
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)
chart_t1

alt.Chart(...)

In [119]:
df_plot = df_sales.loc[(df_sales["Year"] == 2010) & (df_sales["Month"] == 12)]
df_plot = df_plot.groupby("Day").sum()
df_plot["Day"]= df_plot.index

# Visualisierung der Umsätze pro Tag im Dezember 2010
chart_t2 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("Day",
            axis=alt.Axis(title="Dezember 2010", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("TotalPrice", 
            axis=alt.Axis(title = "Umsatz",
                          titleAnchor="middle"))
).properties(
    title={"text":["Umsatz pro Tag"], "subtitle":["Im Dezember 2010"]},
    width=550,
    height=350
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
)
chart_t2

alt.Chart(...)

In [120]:
# Visualisierung der Umsätze pro Tag im Dezember 2011
chart_t3 = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("Day",
            axis=alt.Axis(title="Dezember 2011", 
                          labelAngle=0,
                          titleAnchor="start",
                          grid=False)),
    y=alt.Y("TotalPrice", 
            axis=alt.Axis(title = "Umsatz",
                          titleAnchor="middle"))
).properties(
    title={"text":["Umsatz pro Tag"], "subtitle":["Im Dezember 2011"]},
    width=500,
    height=200
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font='Futura std medium',
    color='black',
    anchor='start'
).configure_axis(
    labelFont='Futura std medium',
    titleFont='Futura std medium',
    labelFontSize=14,
    titleFontSize=18
    
)
chart_t3

alt.Chart(...)

### Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## Conclusion and recommended action

Das Geschäft muss versuchen, sein Warenangebot zu variieren, um ein größeres Spektrum an Kunden aus dem Ausland anzusprechen.

Die Konzentration auf die Erweiterung des internationalen Kundenkreises wird letztlich dazu beitragen, dass das Geschäft weniger von Kunden aus dem Vereinigten Königreich abhängig ist und sich somit stärker diversifizieren kann. Auf diese Weise kann das Geschäft seine Einnahmequellen diversifizieren und seine Abhängigkeit von einer bestimmten Nation und Region (UK / Europa) reduzieren.

Da einige Länder wie Australien und Japan einen hohen durchschnittlichen Warenkorbwert aufweisen, ist es von Vorteil, sich auf den Ausbau der Präsenz in der Region Asien-Pazifik zu konzentrieren. Damit die Präsenz dort gesteigert werden kann muss der Online-Shop eine gezielte Marketing- und Werbekampagne in jedem dieser Länder starten und dabei den Fokus auf die Waren legen, die in diesen Märkten am beliebtesten sind. Eine zusätzliche Investition in Marktforschung ist dabei nötig. Zudem muss ein Zweitstandort für kürzere Lieferwege in der Region Asien-Pazifik eröffnet werden.

### Literature

Alvarez & Marsal. (2021). The shape of retail: the true cost of online. In https://www.alvarezandmarsal.com/.

Statista. (2022a, Mai 11). Prognose der Marktentwicklung des Online-Handels in Deutschland bis 2022. https://de.statista.com/statistik/daten/studie/202905/umfrage/prognostiziertes-marktvolumen-des-deutschen-versandhandels/

Statista. (2022b, Juli 25). Prognose der Umsätze im B2B-E-Commerce in den USA bis 2025. https://de.statista.com/statistik/daten/studie/1321716/umfrage/umsaetze-im-b2b-e-commerce-in-den-usa/